In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-1.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-1        4    2    4    0    2    5     1   0   2  ...  1.000000   
1      prop-1        4   12    4    1   19   44    66   2  18  ...  0.000000   
2      prop-1        4   10    4    0   13   48    45   4  13  ...  1.000000   
3      prop-1        4    2    3    0    5    7     1   0   5  ...  1.000000   
4      prop-1        4    8    1    0    4   38     0   3   1  ...  0.500000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96   9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14   7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0   0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10  26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9  51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0        0  0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1        0  0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2        0  0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3        0  0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4        0  0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[18471 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 4]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...   dam  moa  \
0     prop-1        4    2    4    0    2    5     1   0   2  ...  1.00    0   
1     prop-1        4   12    4    1   19   44    66   2  18  ...  0.00    0   
2     prop-1        4   10    4    0   13   48    45   4  13  ...  1.00    0   
3     prop-1        4    2    3    0    5    7     1   0   5  ...  1.00    0   
4     prop-1        4    8    1    0    4   38     0   3   1  ...  0.50    0   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...   ...  ...   
3017  prop-1        4    7    4    0   18   51    21   0  18  ...  1.00    0   
3018  prop-1        4    3    3    0   27   40     3   0  27  ...  0.00    0   
3019  prop-1        4   10    3    0   19   55    45   0  19  ...  0.20    0   
3020  prop-1        4   30    1    0   58  171   429   9  49  ...  0.00    0   
3021  prop-1        4    4    1    0   10   50     4   0  10  ...  0.75    0   

           mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0     0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1     0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2     0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3     0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4     0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...        ...       ...  ..  ...        ...     ...     ...  ...  
3017  0.647059  0.238095   1    1  55.000000       4  1.5714    0  
3018  0.777778  0.416667   2    2  55.333333       7  2.6667    0  
3019  0.790698  0.333333   2    4  20.800000       4  1.4000    0  
3020  0.000000  0.125490   0    0  75.566667      64  6.0667    0  
3021  0.000000  0.333333   0    0  58.000000      13  3.5000    0  

[3022 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
2338   14    3    4   19   73    43   4  15    7  0.743590  532  0.333333   
1201    4    6    0    8   24     6   0   8    1  1.000000   92  0.000000   
563     5    3    0    7   33     8   0   7    2  0.937500  158  0.750000   
1240    9    3    0   14   27    34   2  14    9  0.982143  164  0.571429   
1213    0    1    0    0    0     0   0   0    0  2.000000    0  0.000000   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
1663    6    1    0    6   31     9   1   5    3  0.800000  136  0.800000   
291     4    5    0    3    8     6   1   2    1  2.000000   19  0.000000   
1698    8    1    0   18   94     6   0  18    2  0.714286  835  0.500000   
415     3    4    1    6    9     3   1   5    1  2.000000   46  0.000000   
2028   11    1    0    4   45    39   0   4    5  0.650000  625  0.833333   

      moa       mfa       cam  ic  cbm         amc  max_cc  avg_cc  
2338    0  0.294118  0.448718   2    3   36.785714       8  3.0714  
1201    0  0.943396  0.375000   3    4   21.750000       5  2.0000  
563     0  0.823529  0.416667   1    2   29.800000       6  2.0000  
1240    0  0.836735  1.000000   0    0   16.444444       2  1.0000  
1213    0  0.000000  0.000000   0    0    0.000000       0  0.0000  
...   ...       ...       ...  ..  ...         ...     ...     ...  
1663    1  0.000000  0.333333   0    0   20.833333       8  2.0000  
291     0  0.971963  1.000000   1    1    3.750000       1  0.7500  
1698    1  0.000000  0.238095   0    0  102.875000      38  6.7500  
415     0  0.958333  0.555556   1    1   14.333333       2  1.3333  
2028    0  0.000000  0.600000   0    0   55.272727      17  2.2727  

[2417 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
1860    4    1    0    3    4     6   3   0    4  2.000000    4  0.000000   
1838   26    2    0    1   27   299   0   1   25  0.965714  178  0.714286   
1523    3    3    0   29   40     3   0  29    1  2.000000  191  0.000000   
812     3    4    0   14   34     3   0  14    2  1.468750  302  0.937500   
143     0    1    0    0    0     0   0   0    0  2.000000    3  0.000000   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
1629   10    1    0   10   59    15   1   9    6  0.444444  409  1.000000   
1129    4    2    0    7   14     4   0   7    2  1.000000   63  0.333333   
203     2    2    0    6    5     1   0   6    2  2.000000   17  0.000000   
148     0    1    0    0    0     0   0   0    0  2.000000   10  0.000000   
67      3    4    0   10   18     3   0  10    1  1.000000   55  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
1860    0  0.000000  0.625000   0    0   0.000000       1  1.0000  
1838    0  0.351351  0.350000   1    5   5.307692       1  0.9231  
1523    0  0.777778  0.523810   1    1  62.666667       5  2.0000  
812     0  0.846154  0.400000   2    2  94.333333       1  0.6667  
143     0  0.000000  0.000000   0    0   0.000000       0  0.0000  
...   ...       ...       ...  ..  ...        ...     ...     ...  
1629    0  0.000000  0.316667   0    0  39.800000      10  3.0000  
1129    0  0.500000  0.555556   1    1  14.000000       1  0.5000  
203     0  0.833333  0.666667   1    2   7.500000       2  1.0000  
148     0  0.000000  0.000000   0    0   0.000000       0  0.0000  
67      0  0.866667  0.500000   1    1  17.000000       2  1.0000  

[605 rows x 20 columns]

In [9]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [10]:
y_train

bug
2338    0
1201    0
563     0
1240    0
1213    0
...   ...
1663    0
291     0
1698    0
415     0
2028    0

[2417 rows x 1 columns]

In [11]:
y_test

bug
1860    0
1838    0
1523    2
812     0
143     0
...   ...
1629    0
1129    0
203     0
148     0
67      0

[605 rows x 1 columns]

In [12]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [13]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [14]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [15]:
y = y_test['bug'].tolist()
y

[0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,


In [16]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_propv4/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
25/25 [==============================] - 0s 2ms/step - loss: 35022.5391 - mse: 35022.5391 - mae: 91.0454 - root_mean_squared_error: 187.1431 - mean_squared_logarithmic_error: 14.5001
Epoch 2/100
25/25 [==============================] - 0s 906us/step - loss: 29920.0332 - mse: 29920.0332 - mae: 83.1598 - root_mean_squared_error: 172.9741 - mean_squared_logarithmic_error: 13.7876
Epoch 3/100
25/25 [==============================] - 0s 1ms/step - loss: 25541.4043 - mse: 25541.4043 - mae: 76.2250 - root_mean_squared_error: 159.8168 - mean_squared_logarithmic_error: 13.1332
Epoch 4/100
25/25 [==============================] - 0s 1ms/step - loss: 21861.7461 - mse: 21861.7441 - mae: 69.7718 - root_mean_squared_error: 147.8572 - mean_squared_logarithmic_error: 12.4844
Epoch 5/100
25/25 [==============================] - 0s 954us/step - loss: 18790.4336 - mse: 18790.4336 - mae: 63.8211 - root_mean_squared_error: 137.0782 - mean_squared_logarithmic_error: 11.8526
Epoch 6/100

25/25 [==============================] - 0s 953us/step - loss: 1.2129 - mse: 1.2129 - mae: 0.1895 - root_mean_squared_error: 1.1013 - mean_squared_logarithmic_error: 0.1010
Epoch 88/100
25/25 [==============================] - 0s 892us/step - loss: 1.1921 - mse: 1.1921 - mae: 0.1885 - root_mean_squared_error: 1.0919 - mean_squared_logarithmic_error: 0.1005
Epoch 89/100
25/25 [==============================] - 0s 954us/step - loss: 1.1738 - mse: 1.1738 - mae: 0.1874 - root_mean_squared_error: 1.0834 - mean_squared_logarithmic_error: 0.1000
Epoch 90/100
25/25 [==============================] - 0s 947us/step - loss: 1.1540 - mse: 1.1540 - mae: 0.1864 - root_mean_squared_error: 1.0743 - mean_squared_logarithmic_error: 0.0996
Epoch 91/100
25/25 [==============================] - 0s 986us/step - loss: 1.1341 - mse: 1.1341 - mae: 0.1853 - root_mean_squared_error: 1.0649 - mean_squared_logarithmic_error: 0.0990
Epoch 92/100
25/25 [==============================] - 0s 949us/step - loss: 1.1155 

25/25 [==============================] - 0s 1ms/step - loss: 0.1907 - mse: 0.1907 - mae: 0.1146 - root_mean_squared_error: 0.4367 - mean_squared_logarithmic_error: 0.0610
Epoch 73/100
25/25 [==============================] - 0s 950us/step - loss: 0.1907 - mse: 0.1907 - mae: 0.1146 - root_mean_squared_error: 0.4367 - mean_squared_logarithmic_error: 0.0610
Epoch 74/100
25/25 [==============================] - 0s 969us/step - loss: 0.1907 - mse: 0.1907 - mae: 0.1146 - root_mean_squared_error: 0.4367 - mean_squared_logarithmic_error: 0.0610
Epoch 75/100
25/25 [==============================] - 0s 925us/step - loss: 0.1907 - mse: 0.1907 - mae: 0.1146 - root_mean_squared_error: 0.4367 - mean_squared_logarithmic_error: 0.0610
Epoch 76/100
25/25 [==============================] - 0s 939us/step - loss: 0.1907 - mse: 0.1907 - mae: 0.1146 - root_mean_squared_error: 0.4367 - mean_squared_logarithmic_error: 0.0610
Epoch 77/100
25/25 [==============================] - 0s 941us/step - loss: 0.1907 - 

25/25 [==============================] - 0s 937us/step - loss: 0.1945 - mse: 0.1945 - mae: 0.1184 - root_mean_squared_error: 0.4410 - mean_squared_logarithmic_error: 0.0616
Epoch 58/100
25/25 [==============================] - 0s 873us/step - loss: 0.1941 - mse: 0.1941 - mae: 0.1183 - root_mean_squared_error: 0.4406 - mean_squared_logarithmic_error: 0.0616
Epoch 59/100
25/25 [==============================] - 0s 980us/step - loss: 0.1938 - mse: 0.1938 - mae: 0.1182 - root_mean_squared_error: 0.4402 - mean_squared_logarithmic_error: 0.0615
Epoch 60/100
25/25 [==============================] - 0s 922us/step - loss: 0.1935 - mse: 0.1935 - mae: 0.1181 - root_mean_squared_error: 0.4398 - mean_squared_logarithmic_error: 0.0615
Epoch 61/100
25/25 [==============================] - 0s 917us/step - loss: 0.1931 - mse: 0.1931 - mae: 0.1181 - root_mean_squared_error: 0.4395 - mean_squared_logarithmic_error: 0.0615
Epoch 62/100
25/25 [==============================] - 0s 967us/step - loss: 0.1928 

25/25 [==============================] - 0s 856us/step - loss: 309.0661 - mse: 309.0662 - mae: 2.3944 - root_mean_squared_error: 17.5803 - mean_squared_logarithmic_error: 0.5998
Epoch 42/100
25/25 [==============================] - 0s 927us/step - loss: 284.5090 - mse: 284.5090 - mae: 2.2366 - root_mean_squared_error: 16.8674 - mean_squared_logarithmic_error: 0.5626
Epoch 43/100
25/25 [==============================] - 0s 946us/step - loss: 260.8983 - mse: 260.8982 - mae: 2.0786 - root_mean_squared_error: 16.1523 - mean_squared_logarithmic_error: 0.5241
Epoch 44/100
25/25 [==============================] - 0s 946us/step - loss: 238.0843 - mse: 238.0843 - mae: 1.9388 - root_mean_squared_error: 15.4300 - mean_squared_logarithmic_error: 0.4912
Epoch 45/100
25/25 [==============================] - 0s 999us/step - loss: 218.8156 - mse: 218.8156 - mae: 1.8183 - root_mean_squared_error: 14.7924 - mean_squared_logarithmic_error: 0.4626
Epoch 46/100
25/25 [==============================] - 0s 9

...

25/25 [==============================] - 0s 1ms/step - loss: 0.2294 - mse: 0.2294 - mae: 0.1211 - root_mean_squared_error: 0.4789 - mean_squared_logarithmic_error: 0.0640
Epoch 90/100
25/25 [==============================] - 0s 1ms/step - loss: 0.2275 - mse: 0.2275 - mae: 0.1210 - root_mean_squared_error: 0.4769 - mean_squared_logarithmic_error: 0.0640
Epoch 91/100
25/25 [==============================] - 0s 1ms/step - loss: 0.2254 - mse: 0.2254 - mae: 0.1208 - root_mean_squared_error: 0.4748 - mean_squared_logarithmic_error: 0.0639
Epoch 92/100
25/25 [==============================] - 0s 943us/step - loss: 0.2237 - mse: 0.2237 - mae: 0.1207 - root_mean_squared_error: 0.4730 - mean_squared_logarithmic_error: 0.0639
Epoch 93/100
25/25 [==============================] - 0s 993us/step - loss: 0.2220 - mse: 0.2220 - mae: 0.1206 - root_mean_squared_error: 0.4712 - mean_squared_logarithmic_error: 0.0638
Epoch 94/100
25/25 [==============================] - 0s 1ms/step - loss: 0.2206 - mse: 0

25/25 [==============================] - 0s 1ms/step - loss: 0.1907 - mse: 0.1907 - mae: 0.1146 - root_mean_squared_error: 0.4367 - mean_squared_logarithmic_error: 0.0610
Epoch 75/100
25/25 [==============================] - 0s 1ms/step - loss: 0.1907 - mse: 0.1907 - mae: 0.1146 - root_mean_squared_error: 0.4367 - mean_squared_logarithmic_error: 0.0610
Epoch 76/100
25/25 [==============================] - 0s 923us/step - loss: 0.1907 - mse: 0.1907 - mae: 0.1146 - root_mean_squared_error: 0.4367 - mean_squared_logarithmic_error: 0.0610
Epoch 77/100
25/25 [==============================] - 0s 924us/step - loss: 0.1907 - mse: 0.1907 - mae: 0.1146 - root_mean_squared_error: 0.4367 - mean_squared_logarithmic_error: 0.0610
Epoch 78/100
25/25 [==============================] - 0s 1ms/step - loss: 0.1907 - mse: 0.1907 - mae: 0.1146 - root_mean_squared_error: 0.4367 - mean_squared_logarithmic_error: 0.0610
Epoch 79/100
25/25 [==============================] - 0s 1ms/step - loss: 0.1907 - mse: 0

25/25 [==============================] - 0s 1ms/step - loss: 39.0355 - mse: 39.0355 - mae: 1.3488 - root_mean_squared_error: 6.2478 - mean_squared_logarithmic_error: 0.5678
Epoch 59/100
25/25 [==============================] - 0s 989us/step - loss: 36.7569 - mse: 36.7569 - mae: 1.2834 - root_mean_squared_error: 6.0627 - mean_squared_logarithmic_error: 0.5406
Epoch 60/100
25/25 [==============================] - 0s 1ms/step - loss: 34.6092 - mse: 34.6092 - mae: 1.2198 - root_mean_squared_error: 5.8830 - mean_squared_logarithmic_error: 0.5139
Epoch 61/100
25/25 [==============================] - 0s 1ms/step - loss: 32.4971 - mse: 32.4971 - mae: 1.1643 - root_mean_squared_error: 5.7006 - mean_squared_logarithmic_error: 0.4913
Epoch 62/100
25/25 [==============================] - 0s 997us/step - loss: 30.7692 - mse: 30.7692 - mae: 1.1113 - root_mean_squared_error: 5.5470 - mean_squared_logarithmic_error: 0.4689
Epoch 63/100
25/25 [==============================] - 0s 1ms/step - loss: 28.97

Epoch 1/100
25/25 [==============================] - 0s 3ms/step - loss: 718.8691 - mse: 718.8690 - mae: 2.4089 - root_mean_squared_error: 26.8117 - mean_squared_logarithmic_error: 0.6070
Epoch 2/100
25/25 [==============================] - 0s 1ms/step - loss: 617.1442 - mse: 617.1442 - mae: 2.1583 - root_mean_squared_error: 24.8424 - mean_squared_logarithmic_error: 0.5492
Epoch 3/100
25/25 [==============================] - 0s 1ms/step - loss: 546.0374 - mse: 546.0374 - mae: 1.9484 - root_mean_squared_error: 23.3674 - mean_squared_logarithmic_error: 0.4989
Epoch 4/100
25/25 [==============================] - 0s 965us/step - loss: 474.4408 - mse: 474.4408 - mae: 1.7648 - root_mean_squared_error: 21.7817 - mean_squared_logarithmic_error: 0.4571
Epoch 5/100
25/25 [==============================] - 0s 1ms/step - loss: 407.4466 - mse: 407.4466 - mae: 1.6107 - root_mean_squared_error: 20.1853 - mean_squared_logarithmic_error: 0.4252
Epoch 6/100
25/25 [==============================] - 0s 1m

25/25 [==============================] - 0s 1ms/step - loss: 0.4051 - mse: 0.4051 - mae: 0.1376 - root_mean_squared_error: 0.6365 - mean_squared_logarithmic_error: 0.0732
Epoch 89/100
25/25 [==============================] - 0s 1ms/step - loss: 0.3998 - mse: 0.3998 - mae: 0.1370 - root_mean_squared_error: 0.6323 - mean_squared_logarithmic_error: 0.0729
Epoch 90/100
25/25 [==============================] - 0s 1ms/step - loss: 0.3941 - mse: 0.3941 - mae: 0.1364 - root_mean_squared_error: 0.6278 - mean_squared_logarithmic_error: 0.0725
Epoch 91/100
25/25 [==============================] - 0s 1ms/step - loss: 0.3892 - mse: 0.3892 - mae: 0.1359 - root_mean_squared_error: 0.6238 - mean_squared_logarithmic_error: 0.0723
Epoch 92/100
25/25 [==============================] - 0s 1ms/step - loss: 0.3843 - mse: 0.3843 - mae: 0.1353 - root_mean_squared_error: 0.6200 - mean_squared_logarithmic_error: 0.0720
Epoch 93/100
25/25 [==============================] - 0s 952us/step - loss: 0.3802 - mse: 0.3

In [17]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_propv4/'

In [18]:
total_training_time

[3.4081597328186035,
 3.2304115295410156,
 3.247006416320801,
 3.302154541015625,
 3.419252634048462,
 3.347914934158325,
 3.3242480754852295,
 3.3987488746643066,
 3.2482566833496094,
 3.229276418685913,
 3.267686605453491,
 3.3833022117614746,
 3.355466365814209,
 3.403768301010132,
 3.4105515480041504,
 3.4406940937042236,
 3.384617805480957,
 3.3317716121673584,
 3.283895254135132,
 3.3335797786712646,
 3.4454293251037598,
 3.403653621673584,
 3.5034236907958984,
 3.5134339332580566,
 3.3613669872283936,
 3.25921630859375,
 3.4992291927337646,
 3.4261536598205566,
 3.433760643005371,
 3.5488409996032715]

In [19]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 3.371509059270223


In [20]:
total_training_loss_history

[{'loss': [35022.5390625,
   29920.033203125,
   25541.404296875,
   21861.74609375,
   18790.43359375,
   16137.4658203125,
   13782.10546875,
   11820.9267578125,
   10164.0625,
   8674.888671875,
   7453.5,
   6347.03076171875,
   5420.5888671875,
   4625.96728515625,
   3929.286376953125,
   2964.12060546875,
   1587.542724609375,
   979.3870239257812,
   609.4069213867188,
   402.5499572753906,
   265.44415283203125,
   190.0345916748047,
   131.68850708007812,
   96.02495574951172,
   70.18128204345703,
   52.9260368347168,
   40.228153228759766,
   31.166147232055664,
   24.817607879638672,
   19.183597564697266,
   15.745162963867188,
   11.859089851379395,
   8.544234275817871,
   7.50830602645874,
   6.7758469581604,
   6.188170909881592,
   5.671670436859131,
   5.326561450958252,
   5.01157808303833,
   4.654959678649902,
   4.387604713439941,
   4.143874168395996,
   3.929736375808716,
   3.718552350997925,
   3.5471251010894775,
   3.390249729156494,
   3.2545578479766846

In [21]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.6622966612378757
avg mae of training: 0.13202694728970527


In [22]:
total_scores

[[5.740253925323486,
  5.740253925323486,
  0.25573575496673584,
  2.3958826065063477,
  0.11549413949251175],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [0.22446422278881073,
  0.22446422278881073,
  0.11475449055433273,
  0.47377654910087585,
  0.06147325411438942],
 [1.8601187467575073,
  1.8601187467575073,
  0.1977802962064743,
  1.3638616800308228,
  0.10227321088314056],
 [0.32946014404296875,
  0.32946014404296875,
  0.13324832916259766,
  0.5739861726760864,
  0.0738687738776207],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [0.40395796298980713,
  0.40395796298980713,
  0.13810433447360992,
  0.6355769038200378,
  0.0770912691950798],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.474123805761337

In [1]:
a = [[5.740253925323486,
  5.740253925323486,
  0.25573575496673584,
  2.3958826065063477,
  0.11549413949251175],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [0.22446422278881073,
  0.22446422278881073,
  0.11475449055433273,
  0.47377654910087585,
  0.06147325411438942],
 [1.8601187467575073,
  1.8601187467575073,
  0.1977802962064743,
  1.3638616800308228,
  0.10227321088314056],
 [0.32946014404296875,
  0.32946014404296875,
  0.13324832916259766,
  0.5739861726760864,
  0.0738687738776207],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [0.40395796298980713,
  0.40395796298980713,
  0.13810433447360992,
  0.6355769038200378,
  0.0770912691950798],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [0.23246021568775177,
  0.23246021568775177,
  0.11601550877094269,
  0.48214128613471985,
  0.06375754624605179],
 [0.32674485445022583,
  0.32674485445022583,
  0.12804736196994781,
  0.571615993976593,
  0.07065793126821518],
 [0.2256755530834198,
  0.2256755530834198,
  0.11386335641145706,
  0.47505322098731995,
  0.062078528106212616],
 [0.22467270493507385,
  0.22467270493507385,
  0.11381712555885315,
  0.47399652004241943,
  0.061586637049913406],
 [0.23776187002658844,
  0.23776187002658844,
  0.14401085674762726,
  0.48760831356048584,
  0.06768336147069931],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [2.0445892810821533,
  2.0445892810821533,
  0.19825805723667145,
  1.4298913478851318,
  0.10093134641647339],
 [0.6695857048034668,
  0.6695857048034668,
  0.14611971378326416,
  0.8182821869850159,
  0.07873833924531937],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [0.46211740374565125,
  0.46211740374565125,
  0.13908512890338898,
  0.6797921657562256,
  0.07180113345384598],
 [0.22480136156082153,
  0.22480136156082153,
  0.11251157522201538,
  0.4741322100162506,
  0.06158275157213211],
 [0.4522688388824463,
  0.4522688388824463,
  0.13577154278755188,
  0.6725093722343445,
  0.06927221268415451],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [0.2861868143081665,
  0.2861868143081665,
  0.12456803768873215,
  0.5349643230438232,
  0.0676715150475502],
 [0.22479338943958282,
  0.22479338943958282,
  0.11239669471979141,
  0.4741238057613373,
  0.06157529726624489],
 [3.3043930530548096,
  3.3043930530548096,
  0.28635573387145996,
  1.8177989721298218,
  0.1468610018491745],
 [0.7038355469703674,
  0.7038355469703674,
  0.1480066329240799,
  0.8389490842819214,
  0.06977616250514984]]

In [2]:
test_mae = []
for i in range(30):
    test_mae.append(a[i][2])

In [3]:
test_mae

[0.25573575496673584,
 0.11239669471979141,
 0.11475449055433273,
 0.1977802962064743,
 0.13324832916259766,
 0.11239669471979141,
 0.11239669471979141,
 0.13810433447360992,
 0.11239669471979141,
 0.11239669471979141,
 0.11239669471979141,
 0.11601550877094269,
 0.12804736196994781,
 0.11386335641145706,
 0.11381712555885315,
 0.14401085674762726,
 0.11239669471979141,
 0.11239669471979141,
 0.19825805723667145,
 0.14611971378326416,
 0.11239669471979141,
 0.13908512890338898,
 0.11251157522201538,
 0.13577154278755188,
 0.11239669471979141,
 0.11239669471979141,
 0.12456803768873215,
 0.11239669471979141,
 0.28635573387145996,
 0.1480066329240799]

In [26]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.6883622959256173
avg mae: 0.13649380579590797
avg are: 0.08048484697937966


In [27]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.1946830078959465
median testing mse: 0.22523845732212067


In [28]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.11668102443218231
median testing mae: 0.1143089234828949


In [29]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.07933884114027023


In [30]:
test_ARE

[0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07735537,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.08429752,
 0.07933884,
 0.08595041,
 0.07933884,
 0.07933884,
 0.07768595,
 0.07933884,
 0.07933884,
 0.105785124]

In [9]:
sum([0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07735537,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.07933884,
 0.08429752,
 0.07933884,
 0.08595041,
 0.07933884,
 0.07933884,
 0.07768595,
 0.07933884,
 0.07933884,
 0.105785124][:10])/10

0.07933884000000001